# Data Preparation and Aggregation

This notebook is responsible for cleaning and processing the datasets from the raw data directory. The goal is to prepare the data for visualization in the D3.js web application.

In [1]:
import pandas as pd
import os

# Define paths
raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'

# Function to load and clean political ads data
def load_and_clean_political_ads(file_name):
    df = pd.read_csv(os.path.join(raw_data_path, file_name))
    # Perform cleaning operations
    df.dropna(inplace=True)  # Example operation: drop missing values
    # Additional cleaning steps can be added here
    return df

# Load and clean datasets
political_ads = load_and_clean_political_ads('political_ads_data.csv')

# Save processed data
political_ads.to_csv(os.path.join(processed_data_path, 'political_ads_cleaned.csv'), index=False)

# Function to load and clean user-data requests
def load_and_clean_user_requests(file_name):
    df = pd.read_csv(os.path.join(raw_data_path, file_name))
    # Perform cleaning operations
    df.dropna(inplace=True)  # Example operation: drop missing values
    # Additional cleaning steps can be added here
    return df

# Load and clean datasets
user_requests = load_and_clean_user_requests('user_requests_data.csv')

# Save processed data
user_requests.to_csv(os.path.join(processed_data_path, 'user_requests_cleaned.csv'), index=False)

# Function to load and clean content removal requests
def load_and_clean_content_removal(file_name):
    df = pd.read_csv(os.path.join(raw_data_path, file_name))
    # Perform cleaning operations
    df.dropna(inplace=True)  # Example operation: drop missing values
    # Additional cleaning steps can be added here
    return df

# Load and clean datasets
content_removal = load_and_clean_content_removal('content_removal_data.csv')

# Save processed data
content_removal.to_csv(os.path.join(processed_data_path, 'content_removal_cleaned.csv'), index=False)

# Summary of processed data
print('Political Ads Data:', political_ads.head())
print('User Requests Data:', user_requests.head())
print('Content Removal Data:', content_removal.head())

ModuleNotFoundError: No module named 'pandas'